In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil
from open3d.pipelines import registration
from predict_maskpls import RefinerModel

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 7

ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]

minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = 'pcd_preprocessed/output_chunks/'


dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

This cell can be ignored after first run as outputs are stored 

Now we subsample the poses based on a voxel_size

In [ ]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored

In [ ]:
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0





#out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/' 
out_kitti_instance = out_chunks + 'dinov2' + str(SEQUENCE_NUM) + '/' 
#out_kitti_instance = out_chunks + 'out_refined/'
        
out_refined = out_chunks + 'out_refined'  + str(SEQUENCE_NUM) + '/' 
if os.path.exists(out_refined) == True : 
        shutil.rmtree(out_refined)
os.makedirs(out_refined)


maskpls = RefinerModel()
eval_refiner = True 

out_data = []

center_ids = os.listdir(out_kitti_instance)
for sequence in range(len(center_ids)):
        if NCUT_ground == False : 
                
                name = center_ids[sequence]
                print("sequence",sequence)
                cur_pcd = o3d.io.read_point_cloud(out_kitti_instance + name)
                predicted_refined = maskpls.forward_and_project(cur_pcd)
                #maskpls.project_pseudo_labels(cur_pcd,center_ids[sequence],name)
                o3d.io.write_point_cloud(out_refined + name, predicted_refined,write_ascii=False, compressed=False, print_progress=False)
                
                        #pcd_dbscan = DBSCAN_clustering_logic(cur_pcd,
                        #                        eps=0.3, min_samples=10)
                        #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                
                
                #kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,kitti_labels['ground']['instance'][sequence].reshape(-1,))
                #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                

        
        
        

In [ ]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds


def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels

def remove_semantics_pcd(pcd,labels,preds):
        cols = np.asarray(pcd.colors)
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
                        cols[pred_idcs] = np.array([0,0,0])
                        
        pcd.colors = o3d.utility.Vector3dVector(cols)   
        return pcd




In [ ]:
out_kitti = out_chunks + 'out_kitti/'
out_kitti_instance = out_chunks + 'out_kitti_instance/'


point_clouds_refined = get_merge_pcds(out_refined)[:-1]

#merge = merge_chunks_unite_instances(point_clouds)
#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
#merge_kitti = merge_unite_gt(point_clouds_kitti)
#merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
merge_refined = merge_chunks_unite_instances(point_clouds_refined)

#o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_refined" + str(SEQUENCE_NUM) + ".pcd", merge_refined, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_dbscan.pcd", merge_dbscan, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti.pcd", merge_kitti, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance.pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)

#unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)



In [ ]:
#new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + "merge_part_kitti_instance" + str(SEQUENCE_NUM) +  ".pcd")
merge_refined = o3d.io.read_point_cloud(out_folder + 'merge_refined' + str(SEQUENCE_NUM) + '.pcd')
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

new_refined = remove_semantics_pcd(merge_refined,labels_kitti,labels_refined)

In [ ]:
unique_colors, labels_refined = np.unique(np.asarray(new_refined.colors),axis=0, return_inverse=True)
label_to_confidence = {}
pcd_cols = np.asarray(new_refined.colors) 
for label in list(np.unique(labels_refined)):
	idcs = np.where(labels_refined == label)[0]
	cur_color = pcd_cols[idcs[0]]
	key = str(int(cur_color[0] * 255)) + '|' + str(int(cur_color[1] * 255)) + '|' + str(int(cur_color[2] * 255))
	label_to_confidence[label] = maskpls.confs_dict[key].cpu().item()

In [ ]:
metrics_refined = Metrics(name='refined')
unique_colors, orig_labels = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

metrics_refined.update_stats(orig_labels,labels_refined,labels_kitti,label_to_confidence,calc_all=True)
